In [1]:
import tensorflow as tf

In [14]:
# params
num_words = 50000
embedding_size = 50
maxlen = 250

In [15]:
# load the dataset
train_data, test_data = tf.keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=num_words,
    skip_top=0,
    maxlen=maxlen,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3,
)

In [16]:
x_train, y_train = train_data
x_test, y_test = test_data

In [19]:
# quickly pad the datasets
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

Define the model

In [24]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Embedding(num_words, embedding_size, input_length=maxlen),
        tf.keras.layers.LSTM(128),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ]
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 50)           2500000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,591,777
Trainable params: 2,591,777
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Train

In [26]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64)

Epoch 1/3
268/268 [==============================] - 18s 65ms/step - loss: 0.5891 - accuracy: 0.6567 - val_loss: 0.3398 - val_accuracy: 0.8584
Epoch 2/3
268/268 [==============================] - 18s 66ms/step - loss: 0.2189 - accuracy: 0.9196 - val_loss: 0.3552 - val_accuracy: 0.8468
Epoch 3/3
268/268 [==============================] - 17s 64ms/step - loss: 0.1232 - accuracy: 0.9608 - val_loss: 0.3768 - val_accuracy: 0.8463


Evaluation just for fun

In [27]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 54.20%
